# 基于GNN的金融异常检测任务

## 1. 实验介绍

反欺诈是金融行业永恒的主题，在互联网金融信贷业务中，数字金融反欺诈技术已经得到广泛应用并取得良好效果，这其中包括了近几年迅速发展并在各个领域
得到越来越广泛应用的图神经网络。本项目以互联网智能风控为背景，从用户相互关联和影响的视角，探索满足风控反欺诈领域需求的，可拓展、高效的图神经
网络应用方案，从而帮助更好地识别欺诈用户。

本项目主要关于实现预测模型(**不限于图神经网络**)，进行节点异常检测任务，并验证模型精度。而本项目基于的数据集[DGraph](https://dgraph.xinye.com/introduction)，[DGraph](https://dgraph.xinye.com/introduction)
是大规模动态图数据集的集合，由真实金融场景中随着时间演变事件和标签构成。

### 1.1 实验目的

- 了解如何使用Pytorch进行神经网络训练
- 了解如何使用Pytorch-geometric等图网络深度学习库进行简单图神经网络设计(推荐使用GAT, GraphSAGE模型)。
- 了解如何利用MO平台进行模型性能评估。

### 1.2 预备知识
- 具备一定的深度学习理论知识，如卷积神经网络、损失函数、优化器，训练策略等。
- 了解并熟悉Pytorch计算框架。
- 学习Pytorch-geometric，请前往：https://pytorch-geometric.readthedocs.io/en/latest/
    
### 1.3实验环境
- numpy = 1.18.5  
- pytorch = 1.4.0  
- torch_geometric = 1.7.0  
- torch_scatter = 2.0.3  
- torch_sparse = 0.5.1  

## 2. 实验内容

### 2.1 数据集信息
DGraph-Fin 是一个由数百万个节点和边组成的有向无边权的动态图。它代表了Finvolution Group用户之间的社交网络，其中一个节点对应一个Finvolution 用户，从一个用户到另一个用户的边表示**该用户将另一个用户视为紧急联系人**。
下面是`位于dataset/DGraphFin目录`的DGraphFin数据集的描述:
```
x:  20维节点特征向量
y:  节点对应标签，一共包含四类。其中类1代表欺诈用户而类0代表正常用户(实验中需要进行预测的两类标签)，类2和类3则是背景用户，即无需预测其标签。
edge_index:  图数据边集,每条边的形式(id_a,id_b)，其中ids是x中的索引
edge_type: 共11种类型的边
edge_timestamp: 脱敏后的时间戳
train_mask, valid_mask, test_mask: 训练集，验证集和测试集掩码
```
本预测任务为识别欺诈用户的节点预测任务,只需要将欺诈用户（Class 1）从正常用户（Class 0）中区分出来。需要注意的是，其中测试集中样本对应的label**均被标记为-100**。

### 2.2 导入相关包

导入相应模块，设置数据集路径、设备等。

In [1]:
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

#设置gpu设备
device = 0
device = 'cpu' #f'cuda:{device}' if torch.cuda.is_available() else 
device = torch.device(device)


d:\SoftWare\Anaconda3\envs\pytorch-cuda115\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2.3 数据处理

在使用数据集训练网络前，首先需要对数据进行归一化等预处理，如下：

In [197]:
path='./datasets/632d74d4e2843a53167ee9a1-momodel/' #数据保存路径
save_dir='./results/' #模型保存路径
dataset_name='DGraph'
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor(remove_edge_index=False))

nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2    #本实验中仅需预测类0和类1

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric() #将有向图转化为无向图


if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  #划分训练集，验证集

train_idx = split_idx['train']
valid_idx = split_idx['valid']
test_idx = split_idx['test']

result_dir = prepare_folder(dataset_name,'mlp')

111


In [199]:
data.edge_index.shape

torch.Size([2, 4300999])

In [3]:
split_idx['train'].shape

torch.Size([857899])

In [118]:
data.adj_t

SparseTensor(row=tensor([      0,       0,       1,  ..., 3700548, 3700548, 3700549]),
             col=tensor([ 826823, 3145251, 1116391,  ..., 2110278, 2609004, 2570092]),
             size=(3700550, 3700550), nnz=7994520, density=0.00%)

这里我们可以查看数据各部分维度

In [4]:
print(data)
print(data.x.shape)  #feature
print(data.y.shape)  #label

Data(x=[3700550, 20], edge_attr=[4300999], y=[3700550], train_mask=[857899], valid_mask=[183862], test_mask=[183840], adj_t=[3700550, 3700550, nnz=7994520])
torch.Size([3700550, 20])
torch.Size([3700550])


### 2.4 定义模型
这里我们使用简单的多层感知机作为例子：

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    图注意力层
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features  # 节点表示向量的输入特征维度
        self.out_features = out_features  # 节点表示向量的输出特征维度
        self.dropout = dropout  # dropout参数
        self.alpha = alpha  # leakyrelu激活的参数
        self.concat = concat  # 如果为true, 再进行elu激活

        # 定义可训练参数，即论文中的W和a
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)  # xavier初始化
        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)  # xavier初始化

        # 定义leakyrelu激活函数
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, inp, adj):
        """
        inp: input_fea [N, in_features]  in_features表示节点的输入特征向量元素个数
        adj: 图的邻接矩阵 维度[N, N] 非零即一，数据结构基本知识
        """
        h = torch.mm(inp, self.W)  # [N, out_features]
        N = h.size()[0]  # N 图的节点数

        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        # [N, N, 2*out_features]
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        # [N, N, 1] => [N, N] 图注意力的相关系数（未归一化）

        zero_vec = -1e12 * torch.ones_like(e)  # 将没有连接的边置为负无穷
        attention = torch.where(adj > 0, e, zero_vec)  # [N, N]
        # 表示如果邻接矩阵元素大于0时，则两个节点有连接，该位置的注意力系数保留，
        # 否则需要mask并置为非常小的值，原因是softmax的时候这个最小值会不考虑。
        attention = F.softmax(attention, dim=1)  # softmax形状保持不变 [N, N]，得到归一化的注意力权重！
        attention = F.dropout(attention, self.dropout, training=self.training)  # dropout，防止过拟合
        h_prime = torch.matmul(attention, h)  # [N, N].[N, out_features] => [N, out_features]
        # 得到由周围节点通过注意力权重进行更新的表示
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class GAT(nn.Module):
    def __init__(self, n_feat, n_hid, n_class, dropout, alpha, n_heads):
        """Dense version of GAT
        n_heads 表示有几个GAL层，最后进行拼接在一起，类似self-attention
        从不同的子空间进行抽取特征。
        """
        super(GAT, self).__init__()
        self.dropout = dropout

        # 定义multi-head的图注意力层
        self.attentions = [GraphAttentionLayer(n_feat, n_hid, dropout=dropout, alpha=alpha, concat=True) for _ in
                           range(n_heads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)  # 加入pytorch的Module模块
        # 输出层，也通过图注意力层来实现，可实现分类、预测等功能
        self.out_att = GraphAttentionLayer(n_hid * n_heads, n_class, dropout=dropout, alpha=alpha, concat=False)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)  # dropout，防止过拟合
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)  # 将每个head得到的表示进行拼接
        x = F.dropout(x, self.dropout, training=self.training)  # dropout，防止过拟合
        x = F.elu(self.out_att(x, adj))  # 输出并激活
        return F.log_softmax(x, dim=1)  # log_softmax速度变快，保持数值稳定


In [9]:
Wh = torch.randn(3,5) # 3个节点，每个节点5个特征
A = torch.randn(3,3) # 注意力系数矩阵
# 邻接矩阵
adj = torch.tensor([[0,1,1],
                    [1,0,0],
                    [1,0,0]])
zero_vec = -9e15*torch.ones_like(A)
attention = torch.where(adj>0, A, zero_vec)
print(attention)
attention = F.softmax(attention, dim=1)
attention

tensor([[-9.0000e+15,  3.6447e-01,  2.7290e-01],
        [-5.0304e-01, -9.0000e+15, -9.0000e+15],
        [-7.1735e-01, -9.0000e+15, -9.0000e+15]])


tensor([[0.0000, 0.5229, 0.4771],
        [1.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000]])

In [21]:
H.shape[0]

3700550

In [24]:

H = H - H.sum(1)
# # 特征求和
rowsum = np.array(H.sum(1),dtype='float32')
print(rowsum.shape)
# # 倒数
# r_inv = np.power(rowsum,-1).flatten()
# # 解决除0问题
# r_inv[np.isinf(r_inv)] = 0.
# # 转换为对角阵
# r_mat_inv = np.diag(r_inv)
# # 对角阵乘以H，得到标准化矩阵
# H = r_mat_inv.dot(H)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

torch.Size([857899, 20])

In [76]:
H=data.x[split_idx['train']]
print(H.min())
rowsum = H.sum(1)
print(rowsum.min())
for i in range (H.shape[0]):
    H[i] = H[i]/rowsum[i]
H.max()

tensor(-2.0435)
tensor(-18.7909)


tensor(36917.3438)

In [34]:
split_idx['test'].shape

torch.Size([183840])

In [95]:
data.adj_t[split_idx['train']]

SparseTensor(row=tensor([     0,      0,      1,  ..., 857897, 857898, 857898]),
             col=tensor([ 100846,  697288, 1487371,  ..., 2056144, 1194573, 3252165]),
             size=(857899, 3700550), nnz=2235182, density=0.00%)

In [49]:
import scipy.sparse as sp
mx = sp.csr_matrix(data.x[split_idx['train']])
rowsum = np.array(mx.sum(1))
r_inv = np.power(rowsum, -1).flatten()
r_inv[np.isinf(r_inv)] = 0.
r_mat_inv = sp.diags(r_inv)

In [51]:
r_mat_inv

<857899x857899 sparse matrix of type '<class 'numpy.float32'>'
	with 857899 stored elements (1 diagonals) in DIAgonal format>

In [142]:
from array import array
import scipy.sparse as sp
# it's neccessary to use sparse-matrix for nomalization
def normalize_adj(mx):
    """Row-normalize """
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt) # need sparse matrix !
    print(type(mx))
    mx.tocsc
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def normalize_features(mx):
    """Row-normalize """
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # need sparse matrix !
    mx = r_mat_inv.dot(mx)
    return mx

In [161]:
data.adj_t[test_idx]

SparseTensor(row=tensor([     0,      0,      1,  ..., 183838, 183839, 183839]),
             col=tensor([1826595, 3339795, 1057328,  ..., 3281558, 1163259, 3163502]),
             size=(183840, 3700550), nnz=479208, density=0.00%)

In [14]:
cuda = torch.cuda.is_available()
cuda

False

In [8]:
#常规label转one-hot向量
def encode_onehot(labels):            #用单位矩阵来构建onehot向量
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in        #单位矩阵
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


In [88]:
oneh = encode_onehot(np.arange(10))
print(oneh)
torch.LongTensor(np.where(oneh)[1]).type_as(torch.LongTensor(1))

[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]


tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [89]:
np.where(oneh)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64))

In [19]:
data.x.shape[1]

20

In [90]:
hidden = 8 
dropout = 0.6
nb_heads = 8 

alpha = 0.2
lr = 0.005
weight_decay = 5e-4
epochs = 10000
patience = 100

cuda = torch.cuda.is_available()

# 实例化模型 
model = GAT(n_feat=data.x.shape[1], 
            n_hid=hidden, 
            n_class=nlabels,
            dropout=dropout, 
            n_heads=nb_heads, 
            alpha=alpha)
# 优化器
optimizer = torch.optim.Adam(model.parameters(), 
                       lr=lr, 
                       weight_decay=weight_decay)
# if cuda:
#     model.cuda()
#     features = features.cuda()
#     adj = adj.cuda()
#     labels = labels.cuda()
#     idx_train = idx_train.cuda()
#     idx_val = idx_val.cuda()
#     idx_test = idx_test.cuda()

# features, adj, labels = Variable(features), Variable(adj), Variable(labels)


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12698792577800 bytes.

In [189]:
adj=data.adj_t.to_scipy()
adj
# zero_vec = -1e12 * torch.ones_like(e)  # 将没有连接的边置为负无穷
# attention = torch.where(adj > 0) 

<3700550x3700550 sparse matrix of type '<class 'numpy.float32'>'
	with 7994520 stored elements in COOrdinate format>

In [193]:
adj = data.adj_t
features = torch.FloatTensor(normalize_features(data.x))
labels = data.y

train_idx = torch.LongTensor(train_idx)
valid_idx = torch.LongTensor(valid_idx)
test_idx = torch.LongTensor(test_idx)

In [194]:
import time

def train(epoch):
    t = time.time()
    # trian
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[train_idx], labels[train_idx])
    acc_train = accuracy(output[train_idx], labels[train_idx])
    loss_train.backward()
    optimizer.step()
        
    # eval
    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[split_idx], labels[split_idx])
    acc_val = accuracy(output[split_idx], labels[split_idx])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [195]:
train(2)

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 438210249680000 bytes.

### 2.5 训练

使用训练集中的节点用于训练模型，并使用验证集进行挑选模型。

In [ ]:
def train(epoch):
    t = time.time()
    # trian
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[train_idx], labels[train_idx])
    acc_train = accuracy(output[train_idx], labels[train_idx])
    loss_train.backward()
    optimizer.step()
        
    # eval
    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[valid_idx], labels[valid_idx])
    acc_val = accuracy(output[valid_idx], labels[valid_idx])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()


def compute_test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


In [10]:
def test(model, data, split_idx, evaluator):
    # data.y is labels of shape (N, )
    with torch.no_grad():
        model.eval()

        losses, eval_results = dict(), dict()
        for key in ['train', 'valid']:
            node_id = split_idx[key]
            
            out = model(data.x[node_id])
            y_pred = out.exp()  # (N,num_classes)
            
            losses[key] = F.nll_loss(out, data.y[node_id]).item()
            eval_results[key] = evaluator.eval(data.y[node_id], y_pred)[eval_metric]

    return eval_results, losses, y_pred

In [11]:
print(sum(p.numel() for p in model.parameters()))  #模型总参数量

model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=para_dict['lr'], weight_decay=para_dict['weight_decay'])
best_valid = 0
min_valid_loss = 1e8

for epoch in range(1,epochs + 1):
    loss = train(model, data, train_idx, optimizer)
    eval_results, losses, out = test(model, data, split_idx, evaluator)
    train_eval, valid_eval = eval_results['train'], eval_results['valid']
    train_loss, valid_loss = losses['train'], losses['valid']

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), save_dir+'/model.pt') #将表现最好的模型保存

    if epoch % log_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_eval:.3f}, ' # 我们将AUC值乘上100，使其在0-100的区间内
              f'Valid: {100 * valid_eval:.3f} ')

2946
Epoch: 10, Loss: 0.0876, Train: 63.245, Valid: 63.668 
Epoch: 20, Loss: 0.0939, Train: 67.276, Valid: 67.213 
Epoch: 30, Loss: 0.0751, Train: 69.646, Valid: 69.094 
Epoch: 40, Loss: 0.0655, Train: 68.973, Valid: 68.822 
Epoch: 50, Loss: 0.0654, Train: 69.272, Valid: 68.871 
Epoch: 60, Loss: 0.0645, Train: 70.279, Valid: 69.782 
Epoch: 70, Loss: 0.0645, Train: 70.677, Valid: 70.147 
Epoch: 80, Loss: 0.0643, Train: 70.765, Valid: 70.201 
Epoch: 90, Loss: 0.0642, Train: 70.967, Valid: 70.355 
Epoch: 100, Loss: 0.0641, Train: 71.163, Valid: 70.517 
Epoch: 110, Loss: 0.0640, Train: 71.315, Valid: 70.636 
Epoch: 120, Loss: 0.0640, Train: 71.427, Valid: 70.724 
Epoch: 130, Loss: 0.0639, Train: 71.547, Valid: 70.805 
Epoch: 140, Loss: 0.0639, Train: 71.641, Valid: 70.881 
Epoch: 150, Loss: 0.0638, Train: 71.730, Valid: 70.949 
Epoch: 160, Loss: 0.0638, Train: 71.805, Valid: 71.004 
Epoch: 170, Loss: 0.0638, Train: 71.872, Valid: 71.047 
Epoch: 180, Loss: 0.0638, Train: 71.933, Valid: 71.0

### 2.6 模型预测

In [12]:
model.load_state_dict(torch.load(save_dir+'/model.pt')) #载入验证集上表现最好的模型
def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        out = model(data.x[node_id])
        y_pred = out.exp()  # (N,num_classes)
        
    return y_pred

In [13]:
dic={0:"正常用户",1:"欺诈用户"}
node_idx = 0
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

node_idx = 1
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

tensor([0.9965, 0.0035])
节点 0 预测对应的标签为:0, 为正常用户。
tensor([0.9733, 0.0267])
节点 1 预测对应的标签为:0, 为正常用户。


## 3. 作业评分

**作业要求**：    
                         
1. 请加载你认为训练最佳的模型（不限于图神经网络)
2. 提交的作业包括【程序报告.pdf】和代码文件。

**注意：**
          
1. 在训练模型等过程中如果需要**保存数据、模型**等请写到 **results** 文件夹，如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请务必将模型保存在 **results** 文件夹下。
2. 训练出自己最好的模型后，先按照下列 cell 操作方式实现 NoteBook 加载模型测试；请测试通过在进行【系统测试】。
3. 点击左侧栏`提交作业`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
4. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
5. 请加载你认为训练最佳的模型，即请按要求填写**模型路径**。
6. `predict()`函数的输入和输出请不要改动。

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [14]:
## 生成 main.py 时请勾选此 cell
from utils import DGraphFin
from utils.evaluator import Evaluator
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.data import Data
import numpy as np
import os

nlabels = 2    #本实验中仅需预测类0和类1
init_params = {
    'num_layers': 2,
    'hidden_channels': 128,
    'dropout': 0.0,
    'batchnorm': False
}
class MLP(torch.nn.Module):
    def __init__(self
                 , in_channels
                 , hidden_channels
                 , out_channels
                 , num_layers
                 , dropout
                 , batchnorm=True):
        super(MLP, self).__init__()
        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.batchnorm = batchnorm
        if self.batchnorm:
            self.bns = torch.nn.ModuleList()
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            if self.batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        if self.batchnorm:
            for bn in self.bns:
                bn.reset_parameters()

    def forward(self, x):    
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            if self.batchnorm:
                x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return F.log_softmax(x, dim=-1)
    
def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    model = MLP(in_channels=data.x.size(-1), out_channels=nlabels,**init_params)
    # 这里可以加载你的模型
    model.load_state_dict(torch.load('./results/model.pt'))
    # 模型预测时，测试数据已经进行了归一化处理
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        out = model(data.x[node_id])
        y_pred = out.exp()  # (N,num_classes)
    
    return y_pred

111


In [19]:
init_param = {
    'num_layers': 2,
    'hidden_channels': 128,
    'dropout': 0.0,
    'batchnorm': False
}
print(*init_param)

num_layers hidden_channels dropout batchnorm
